1. 필요한 라이브러리나 모듈 import

In [1]:
import random
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow_datasets as tfds

2. tensorflow_datasets 목록
    + 본래 amazon_us_reviews를 다루려고 했으나 403 forbidden 메세지를 보아 더이상 tensorflow에서 지원하지 않는다는 것을 확인
    + 이에 따라 대체 데이터셋으로 beans를 이용하기로 결정

In [2]:
tfds.list_builders()

['abstract_reasoning',
 'accentdb',
 'aeslc',
 'aflw2k3d',
 'ag_news_subset',
 'ai2_arc',
 'ai2_arc_with_ir',
 'amazon_us_reviews',
 'anli',
 'answer_equivalence',
 'arc',
 'asqa',
 'asset',
 'assin2',
 'bair_robot_pushing_small',
 'bccd',
 'beans',
 'bee_dataset',
 'beir',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'ble_wind_field',
 'blimp',
 'booksum',
 'bool_q',
 'bucc',
 'c4',
 'c4_wsrs',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cardiotox',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'cherry_blossoms',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar100_n',
 'cifar10_1',
 'cifar10_corrupted',
 'cifar10_n',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'clic',
 'clinc_oos',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coco_captions',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'common_voice',
 'conll2002',
 'conll2003',
 'controlled_noisy_web_labels'

3. train 데이터

In [3]:
tfds_name = 'beans'

train_dataset, train_info = tfds.load(name=tfds_name, split=tfds.Split.TRAIN, with_info=True)
test_dataset, test_info = tfds.load(name=tfds_name, split=tfds.Split.TEST, with_info=True)

In [4]:
print(train_dataset)
print(test_dataset)

<_PrefetchDataset element_spec={'image': TensorSpec(shape=(500, 500, 3), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>
<_PrefetchDataset element_spec={'image': TensorSpec(shape=(500, 500, 3), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>


In [5]:
print(train_info)
print(test_info)

tfds.core.DatasetInfo(
    name='beans',
    full_name='beans/0.1.0',
    description="""
    Beans is a dataset of images of beans taken in the field using smartphone
    cameras. It consists of 3 classes: 2 disease classes and the healthy class.
    Diseases depicted include Angular Leaf Spot and Bean Rust. Data was annotated by
    experts from the National Crops Resources Research Institute (NaCRRI) in Uganda
    and collected by the Makerere AI research lab.
    """,
    homepage='https://github.com/AI-Lab-Makerere/ibean/',
    data_path='C:\\Users\\user\\tensorflow_datasets\\beans\\0.1.0',
    file_format=tfrecord,
    download_size=171.69 MiB,
    dataset_size=171.63 MiB,
    features=FeaturesDict({
        'image': Image(shape=(500, 500, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=3),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=128, num_shards=1>,
        '

4. seed(난수)를 생성한다

In [6]:
value = 42
tf.random.set_seed(value)
np.random.seed(value)

5. train_dataset을 list로 전환하여 train_x, train_y를 만든다.
    test_dataset에도 마찬가지로 적용한다.

In [7]:
# 열 정보 확인
column_names = list(train_info.features.keys())
print(column_names)

['image', 'label']


In [8]:
# train_x, train_y 만들기

datalist = list(train_dataset.as_numpy_iterator())

train_x = np.array([data['image'] for data in datalist])
train_y = np.array([data['label'] for data in datalist])

print(train_x)
print(train_y)

[[[[149 132 106]
   [141 124  98]
   [134 118  95]
   ...
   [129 152  64]
   [129 152  64]
   [130 153  65]]

  [[151 134 108]
   [140 124  98]
   [133 117  94]
   ...
   [126 149  61]
   [127 150  62]
   [127 150  62]]

  [[150 134 108]
   [140 124  98]
   [133 117  94]
   ...
   [126 151  60]
   [125 150  59]
   [122 147  56]]

  ...

  [[ 20  33  49]
   [ 21  34  50]
   [ 23  36  52]
   ...
   [149 211 112]
   [145 204 120]
   [142 200 123]]

  [[ 21  34  50]
   [ 22  35  51]
   [ 23  36  52]
   ...
   [151 213 114]
   [142 201 117]
   [136 194 118]]

  [[ 21  34  50]
   [ 21  34  50]
   [ 22  35  51]
   ...
   [154 216 117]
   [141 200 118]
   [133 191 115]]]


 [[[ 97  88  57]
   [ 97  86  56]
   [105  91  64]
   ...
   [126  98  86]
   [131 101  91]
   [121  91  81]]

  [[100  91  60]
   [101  90  60]
   [103  89  60]
   ...
   [125  97  85]
   [121  93  82]
   [119  91  80]]

  [[101  92  59]
   [ 98  87  55]
   [100  86  57]
   ...
   [115  88  77]
   [107  80  69]
   [112  85

In [9]:
# test_x, test_y 만들기

datalist = list(test_dataset.as_numpy_iterator())

test_x = np.array([data['image'] for data in datalist])
test_y = np.array([data['label'] for data in datalist])

print(test_x)
print(test_y)

[[[[141 152  96]
   [157 171 112]
   [153 171 111]
   ...
   [135 113  64]
   [135 113  64]
   [125 103  54]]

  [[132 140  91]
   [133 143  90]
   [163 178 119]
   ...
   [119  97  48]
   [117  95  46]
   [133 111  62]]

  [[133 137 100]
   [107 114  70]
   [136 149  93]
   ...
   [112  88  42]
   [117  93  47]
   [132 108  62]]

  ...

  [[159 128  97]
   [153 122  91]
   [148 117  86]
   ...
   [171 138  97]
   [160 127  86]
   [148 115  74]]

  [[161 130  99]
   [149 118  87]
   [139 108  77]
   ...
   [169 136  95]
   [158 125  84]
   [148 115  74]]

  [[159 128  97]
   [144 113  82]
   [132 101  70]
   ...
   [165 132  91]
   [157 124  83]
   [152 119  78]]]


 [[[102  77  46]
   [ 97  72  41]
   [103  79  45]
   ...
   [ 30  28  15]
   [ 29  27  12]
   [ 29  27  12]]

  [[108  83  52]
   [118  93  62]
   [122  98  64]
   ...
   [ 20  18   5]
   [ 23  21   6]
   [ 26  24   9]]

  [[121  96  65]
   [125 100  69]
   [137 112  81]
   ...
   [ 27  23  11]
   [ 26  22  10]
   [ 26  22

6. 데이터 전처리
    이미지 파일이라서 255.0으로 나누어 0에서 1 사이의 값으로 scaling하고
    이후 one-hot-encoding을 진행하였다 (to_categorical)

In [10]:
train_x = train_x.astype('float32') / 255.0
test_x = train_x.astype('float32') / 255.0

train_y = tf.keras.utils.to_categorical(train_y)
test_y = tf.keras.utils.to_categorical(test_y)

print(train_x)
print(train_y)

[[[[0.58431375 0.5176471  0.41568628]
   [0.5529412  0.4862745  0.38431373]
   [0.5254902  0.4627451  0.37254903]
   ...
   [0.5058824  0.59607846 0.2509804 ]
   [0.5058824  0.59607846 0.2509804 ]
   [0.50980395 0.6        0.25490198]]

  [[0.5921569  0.5254902  0.42352942]
   [0.54901963 0.4862745  0.38431373]
   [0.52156866 0.45882353 0.36862746]
   ...
   [0.49411765 0.58431375 0.23921569]
   [0.49803922 0.5882353  0.24313726]
   [0.49803922 0.5882353  0.24313726]]

  [[0.5882353  0.5254902  0.42352942]
   [0.54901963 0.4862745  0.38431373]
   [0.52156866 0.45882353 0.36862746]
   ...
   [0.49411765 0.5921569  0.23529412]
   [0.49019608 0.5882353  0.23137255]
   [0.47843137 0.5764706  0.21960784]]

  ...

  [[0.07843138 0.12941177 0.19215687]
   [0.08235294 0.13333334 0.19607843]
   [0.09019608 0.14117648 0.20392157]
   ...
   [0.58431375 0.827451   0.4392157 ]
   [0.5686275  0.8        0.47058824]
   [0.5568628  0.78431374 0.48235294]]

  [[0.08235294 0.13333334 0.19607843]
   [0.0

7. 모델 작성

In [ ]:
height = 500
width = 500
channels = 3

model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(64, input_shape=(height * width, channels), return_sequences=True))
model.add(tf.keras.layers.LSTM(64, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

8. 모델 compile

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

9. 모델 fit

In [ ]:
res = model.fit(train_x, train_y, epochs=10, batch_size=100,
                validation_split=0.2, verbose=1)